# Notebook to Train Custom YOLO Models

This notebook is designed to help you train your own YOLO models from scratch or fine-tune existing ones.  
You can use either:

- **Local datasets** (e.g., in YOLO format stored on your Google Drive or GitHub)
- **Datasets from Roboflow**, which can be easily imported via a download link
- **Example dataset**, from a linked github repository

The workflow includes:
- Loading and organizing your dataset
- Writing a custom `.yaml` config file
- Launching training with the `ultralytics` YOLO implementation
- (Optional) Exporting and evaluating your trained model

This is ideal for training models on custom objects — whether you're working with animals, vehicles, tools, or underwater footage.

---

Make sure your dataset is in the correct YOLO structure:

```
dataset/
├── train/
│   ├── images/
│   └── labels/
├── valid/
│   ├── images/
│   └── labels/
├── test/   # optional
│   ├── images/
│   └── labels/
└── data.yaml
```

# Import libraries

In [ ]:
import os
import random
import shutil
import math
import glob
from IPython.display import Image, display
import numpy as np
import time  # Import the time module
from google.colab import runtime
from google.colab import drive
from pathlib import Path
import zipfile
import platform
import gdown
import sys
from pathlib import Path
import sys

from __future__ import annotations
import os, shutil, random, math
from tempfile import mkdtemp
from typing import Optional, Tuple, Dict, List, Sequence

from collections import Counter, defaultdict



In [ ]:
!nvidia-smi

In [ ]:
# Pip install method (recommended)

!pip install 'ultralytics'
# !pip install ultralytics==8.3.195
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO
from IPython.display import display, Image

##tiling
!pip install --upgrade git+https://github.com/Jordan-Pierce/yolo-tiling.git
import sys
sys.path.append('/content/yolo-tiling')

from yolo_tiler import YoloTiler, TileConfig, TileProgress


In [ ]:
### Clone the folder with the functions
!git clone https://github.com/albiangela/train-custom-YOLO-Colab.git
import sys
sys.path.append("/content/train-custom-YOLO-Colab/utils")

### 🔗 Connect to Your Google Drive

Google Colab is a cloud-based Python environment that lets you run code in your browser, with free access to GPUs.  

To access your datasets or save model outputs, you’ll need to connect Colab to your Google Drive.
This allows you to read and write files directly from your Drive, making it easier to store large datasets or export trained models.

Run the cell below to authorize access.

In [ ]:
# Mount google drive
drive.mount("/content/drive/")

# Load data

Use the interactive widget below to choose your dataset source. You can select one of:

### 1. Roboflow
- Paste your **Roboflow API key**, **workspace**, **project**, **version**, and **export format** (e.g., `yolov11`).
- Tip: Usually copied from **Roboflow → Export → Show download code** on your project page.

### 2. Google Drive link (shared `.zip`)
- Copy the **Drive share URL** of your `.zip` (set sharing to **Anyone with the link**).
- Paste it into the widget and confirm to download and extract.

### 3. Example dataset from GitHub (Hexbugs)
- Select **Hexbugs** to load a small, YOLO-formatted example dataset for quick testing.

In [ ]:
# Select and fetch dataset via interactive UI


# workspace_root = Path('/content/drive/MyDrive/Colab Notebooks')
workspace_root = Path('/content/train-custom-YOLO-Colab/')
if workspace_root.exists() and str(workspace_root) not in sys.path:
    sys.path.append(str(workspace_root))
if str(Path.cwd()) not in sys.path:
    sys.path.append(str(Path.cwd()))

from utils.datasets import launch_dataset_selector
workspace_dataset_root = Path('/content/datasets')
launch_dataset_selector(globals(), dataset_root=workspace_dataset_root)



In [ ]:
# workspace_root = Path('/content/drive/MyDrive/Colab Notebooks')
# if workspace_root.exists() and str(workspace_root) not in sys.path:
#     sys.path.append(str(workspace_root))
# if str(Path.cwd()) not in sys.path:
#     sys.path.append(str(Path.cwd()))

from utils.prep import (
    Data,
    auto_select_allowed_ids,
    build_collapse_map,
    build_new_class_ids_from_yaml,
    check_dataset,
    count_labels,
    filter_labels,
    make_data_yaml,
    prepare_yolo_dataset,
    simplify_labels,
    summarize_classes,
    tile_with_yolo_tiler,
)

dataset = Data('/content/datasets/', name)

assert os.path.exists(dataset.location + name + '/train')

dataset.location


### Optional: Rename your annotated labels — What this cell does

- **Purpose:** Collapse/rename your original classes into broader groups (e.g., many ray species → `sting_ray`, many shark types → `shark`) before training.

- **Example**
If you have a training dataset with 4 annotated species (e.g. 'cowtail_sting_ray','pink_stingray','blacktip_reef_shark','whitetip_reef_shark) but only want to train a simple model with two classes such as 'shark' and 'ray', then you can rename your classes to reduce the number of labels. In this case, depending on the class number that was assigned to your species on your annotations dataset (e.g. 0 is 'cowtail_sting_ray', 1 is 'pink_stingray',2 is 'blacktip_reef_shark',3 is 'whitetip_reef_shark), then you can remap as in the example below.

- **`collapse_map`**  
  Maps **original class IDs** → **new group names**.  
  - Only IDs listed here are **kept**.  
  - Commented-out lines are **ignored** (those classes will be dropped).  


- **`allowed_ids`**  
  Set of original IDs you’re keeping (i.e., the keys of `collapse_map`). Used to **filter** annotations.

- **`new_class_ids`**  
  Assigns **final numeric IDs** to the new groups (e.g., `shark:0`, `sting_ray:1`). These become your **contiguous class indices** used by YOLO.

- **Outcome**  
  After your relabel step runs, annotations are remapped so that all sharks share ID `0`, all sting rays share ID `1`, and unlisted classes are dropped.

- **Don’t forget**  
  Update your `data.yaml` to match the **new class list and order** (e.g., `names: ['shark','sting_ray']`).



In [ ]:
# ----------------------------------------------
# 🔁 Step 1: Collapse or remap class labels (optional)
# ----------------------------------------------
# OPTIONAL — Use this to change how labels are grouped, e.g. merging multiple shark types into one class.

# Collapse original class IDs into broader categories
collapse_map = {
    0: 'sting_ray',    # cowtail_sting_ray
    1: 'sting_ray',    # pink_Stingray
    2: 'shark',        # blacktip_reef_shark
    3: 'shark',        # whitetip_reef_shark

}

allowed_ids = set(collapse_map.keys())

# Assign new numeric IDs to the collapsed categories
new_class_ids = {
    'shark': 0,
    'sting_ray': 1,
}

# Auto-select classes, remap IDs, and create a dataset

## What this cell does
- **Auto-selects** classes that meet minimum data thresholds.
- **Builds** a contiguous ID mapping (old → new).
- **Prepares** a filtered, optionally rebalanced **train/val/test** split.

## Steps

### 1) Point to your dataset
- `dataset_root = "/content/datasets/" + name` — the dataset folder chosen via the widget.

### 2) Auto-select viable classes
- `auto_select_allowed_ids(dataset_root, min_instances=40, min_files=10)` returns:
  - `allowed_ids`: original class IDs that pass thresholds.
  - `instance_counts`: total labeled objects per class.
  - `file_counts`: images containing each class.
- Tune thresholds to your data size:
  - `min_instances=40` (min total objects per class).
  - `min_files=10` (min images per class; set `None` to ignore).

### 3) Build a compact remap
- If `allowed_ids` is non-empty:
  - `collapse_map = build_collapse_map(allowed_ids)` → e.g. `{3:0, 4:1, 7:2}`.
  - `new_class_ids = sorted(set(collapse_map.values()))` → e.g. `[0, 1, 2]`.
- Else: `collapse_map = None`, `new_class_ids = None` (skips remap).

### 4) Prepare the filtered dataset
- `prepare_yolo_dataset(...)` runs with:
  - `out_dir = dataset_root + "-filtered_split"` — output folder.
  - `do_change_labels=True` + `collapse_map/new_class_ids` — apply remap to contiguous IDs.
  - `allowed_ids=...`, `drop_others=True` — keep only chosen classes; drop the rest.
  - `prune_empty_fraction=0.9` — remove up to **90%** of empty images (keeps ~10% negatives).
  - `do_tile=False` — skip tiling in this pass.
  - `do_rebalance=True` — mitigate class imbalance across splits.
  - `split=(0.7, 0.2, 0.1)` — train/val/test ratios.
  - `remove_test=False` — keep a test split.

## Output
- A cleaned dataset at `...-filtered_split/` with:
  - `train/`, `val/`, `test/` (images/labels),
  - labels remapped to contiguous IDs,
  - and an updated `data.yaml` compatible with YOLO.

## Notes
- If `allowed_ids` is empty, relax `min_instances`/`min_files` or inspect class distribution.
- Keeping a small fraction of negatives (`prune_empty_fraction`) usually improves generalization.

In [ ]:

dataset_root = "/content/datasets/" + name           # or before splitting

allowed_ids, instance_counts, file_counts = auto_select_allowed_ids(
    dataset_root,
    min_instances=40,   # tune these to your dataset size
    min_files=10        # optional; set None to ignore
)

# 2) Build the class remapping (old → new contiguous ids)
if allowed_ids:
    collapse_map = build_collapse_map(allowed_ids)          # e.g. {3:0, 4:1, 7:2}
    new_class_ids = sorted(set(collapse_map.values()))      # e.g. [0,1,2]
else:
    collapse_map, new_class_ids = None, None

# 2) If you like the selection, run your prep with filtering only (no remap)
if allowed_ids:
    prepare_yolo_dataset(
        dataset_path=dataset_root,
        out_dir=dataset_root + "-filtered_split",
        do_change_labels=True,
        allowed_ids=allowed_ids,
        collapse_map=collapse_map,      # ← now active
        new_class_ids=new_class_ids,    # ← now active
        drop_others=True,               # drop unwanted ids
        prune_empty_fraction=0.9,
        do_tile=False,
        do_rebalance=True,
        split=(0.7, 0.2, 0.1),
        remove_test=False,
    )

### Check number of labels per class

In [ ]:
### Check here for each folder in the newsly created dataset '-filtered_split' how many labels per class we have
out_dir = '/content/datasets/' + name + '-filtered_split'  # e.g., the same `out_dir` you passed to prepare_yolo_dataset
check_dataset(out_dir)


## Optional: crete a new yaml file
If you didn't change labels you can directly copy the yaml file from the original folder to the rebalanced_data folder

In [ ]:
make_yaml = True      # set True to generate a new YAML
copy_yaml = False     # keep False to avoid overwriting

base_dir = "/content/datasets/"           # root folder

src_yaml = os.path.join(base_dir, name, 'data.yaml')
dst_yaml = os.path.join(out_dir, 'data.yaml')

if make_yaml:
    new_class_ids = build_new_class_ids_from_yaml(
        src_yaml=src_yaml,
        allowed_ids=allowed_ids,
        collapse_map=collapse_map,
    )
    yaml_path = make_data_yaml(
        dataset_root=out_dir,
        new_class_ids=new_class_ids,
        has_test=None,  # auto-detect from folder existence
    )
    print('✅ data.yaml written to:', yaml_path)
elif copy_yaml:
    if os.path.exists(src_yaml):
        shutil.copy2(src_yaml, dst_yaml)
        print(f'✅ Copied data.yaml from {src_yaml} → {dst_yaml}')
    else:
        print(f'❌ Source data.yaml not found at {src_yaml}')


### Optional: Create a zip folder with the new dataset to dowload it

In [ ]:
make_zip = True                 # set to False to skip zipping

# === PATHS ===
folder = os.path.join(base_dir, f"{name}-filtered_split")
zip_path = os.path.join(base_dir, f"{name}-filtered_split.zip")

# === CONDITIONAL ZIP ===
if make_zip:
    if os.path.exists(folder):
        if not os.path.exists(zip_path):
            !zip -r -q "{zip_path}" "{folder}"
            print(f"✅ Zipped: {zip_path}")
        else:
            print(f"⚠️ Zip file already exists: {zip_path}")
    else:
        print(f"❌ Folder not found: {folder}")
else:
    print("⏭️ Skipping ZIP creation (make_zip=False)")

### Define output path

In [ ]:
### Change path to your folder
REMOTE_URL = "/content/drive/MyDrive/models/" + name
HOME = "/content/datasets/"

# Change to HOME directory
%cd {HOME}

# Import os and create the folder if it doesn't exist
import os

if not os.path.exists(REMOTE_URL):
    os.makedirs(REMOTE_URL)
    print(f"Directory '{REMOTE_URL}' created.")
else:
    print(f"Directory '{REMOTE_URL}' already exists.")

# Training: Parameters

In [ ]:
# Change to home directory
%cd {HOME}

# ---- User-defined Settings ----
resolution = 1080                # Image resolution for training
epochs = 300                     # Number of training epochs
batch_size = 6                   # Batch size
base_model = "yolo26s"         # Choose model variant. Options: "yolo11n-pose", "yolo11n-seg", "yolo11n" etc.


# ---- Auto-detect task type ----
if "-seg" in base_model:
    task = "segment"
elif "-pose" in base_model:
    task = "pose"
else:
    task = "detect"

# ---- 🔧 Training Settings ----
common_settings = {
    "translate": 0.05,       # Maximum image translation as data augmentation (in % of image size)
    "mixup": 0.1,          # MixUp blending factor for image mixing (usually low for object detection)
    "copy_paste": 0.3,      # Probability of using Copy-Paste augmentation (object pasting)
    "scale": 0.3,           # Random scaling of images for augmentation
    "mosaic": 0.5,             # Enable Mosaic augmentation (combines 4 images into 1)
    "close_mosaic": 10,      # Number of epochs before disabling mosaic for better fine-tuning
    "line_width": 1,         # Line width for label visualization
    "nms": True,             # Apply Non-Maximum Suppression during inference
    "plots": True,           # Save training plots (loss, mAP, etc.)
    "cache": "disk",         # Caching mode: "disk" to speed up I/O
    "single_cls": False,     # If True, treat all objects as one class (for class-agnostic detection)
    "amp": True,             # Enable automatic mixed precision (reduces memory, speeds up training)
    "augment": True,        # If True, applies augmentation at inference time
    "workers": 16,            # Number of dataloader workers (adjust depending on your CPU)
    "multi_scale": False,
    "hsv_h": 0.015,
    "hsv_s": 0.5,
    "hsv_v": 0.4

}


# Modify task-specific augmentations
if task == "detect":
    common_settings.update({
        "degrees": 10,       # Allow full rotation
        "flipud": 0.0,       # Vertical flip probability
        "fliplr": 0.0        # Horizontal flip probability
    })
else:
    common_settings.update({
        "degrees": 0,         # No rotation for pose/seg
        "flipud": 0.0,
        "fliplr": 0.0
    })

# Print CLI training parameters
parms = " ".join([f"{k}={v}" for k, v in common_settings.items()])
print("🔧 Training params:", parms)

# ---- 🗂️ Model Output Naming ----
from datetime import datetime
now = datetime.now()
date_string = now.strftime("%Y-%m-%d-%H") + "_" + dataset.name.replace(" ", "-") + "-" + str(dataset.version)

project = f"{resolution}-{base_model}"
if common_settings["mosaic"] > 0:
    project += "-mosaic"

# if sharkcam:
#     project += "-sharkcam"  # Add logic if needed

# ---- 🧠 Model Weights Source ----
model = base_model  # or path to a pretrained model
print(f"🧪 resolution={resolution} | project={project} | date_string={date_string}")
print(f"📦 model={model} | base_model={base_model} | task={task}")

# ---- 🔒 Safety Check ----
import os
assert model == base_model or os.path.exists(model + ".pt"), f"Model path not found: {model}.pt"

# Training: Run Command

In [ ]:
    # Change to your working directory
%cd {HOME}

# ---- Launch YOLO training ----
yolo_cmd = f"""
yolo task={task} \
     mode=train \
     resume=False \
     model={model}.pt \
     data={out_dir}/data.yaml \
     device=0 \
     name={date_string} \
     project={project} \
     epochs={epochs} \
     imgsz={resolution} \
     batch={batch_size} \
     patience=0 \
     visualize=True \
     {parms}
"""

# ▶️ Run the command
!{yolo_cmd}

#5) Locate last trained model

In [ ]:
# Change to the working directory
%cd {HOME}

runs_root = f"{dataset.location}/runs/{task}/{project}"

# List all subdirectories in the project folder
all_subdirs = [os.path.join(runs_root, d) for d in os.listdir(runs_root)]
all_subdirs = [d for d in all_subdirs if os.path.isdir(d)]

# Keep only those that contain a trained model
all_subdirs = [d for d in all_subdirs if os.path.exists(os.path.join(d, "weights", "last.pt"))]


# Get the most recently modified subdirectory
latest_subdir = max(all_subdirs, key=os.path.getmtime)

# Construct the full path to the latest run
# full_path = HOME + "/" + latest_subdir

print(project)
print(latest_subdir)
# print(full_path)

# Save training parameters to a parms.txt
!echo "{parms}" > {latest_subdir}/parms.txt

### Select and Save Best YOLO Model Based on mAP Metrics

In [ ]:


# Check if the best model weights file exists
print(os.path.exists(os.path.join(latest_subdir, "weights", "best.pt")))

# Load training results CSV
csv = pd.read_csv(os.path.join(latest_subdir, "results.csv"))

# Strip whitespace from column names
csv.columns = [c.strip() for c in csv.columns]

# Compute best epoch score (seg or det)
if "metrics/mAP50-95(M)" in csv.columns:
    combined = (csv["metrics/mAP50-95(M)"] * 0.9 + csv["metrics/mAP50(M)"] * 0.1) + \
               (csv["metrics/mAP50-95(B)"] * 0.9 + csv["metrics/mAP50(B)"] * 0.1)
    index = combined.argmax()
    best_map50_95 = float(csv["metrics/mAP50-95(M)"].values[index])
    best_map50 = float(csv["metrics/mAP50(M)"].values[index])
else:
    combined = (csv["metrics/mAP50-95(B)"] * 0.9 + csv["metrics/mAP50(B)"] * 0.1)
    index = combined.argmax()
    best_map50_95 = float(csv["metrics/mAP50-95(B)"].values[index])
    best_map50 = float(csv["metrics/mAP50(B)"].values[index])

# Copy best.pt to /content with informative name
from_path = os.path.join(latest_subdir, "weights", "best.pt")
to_name = f"{project}-{date_string}-mAP5095_{best_map50_95:.5f}-mAP50_{best_map50:.5f}.pt"
to_path = os.path.join("/content", to_name)

print("copying from", from_path, "to", to_path)
!cp "{from_path}" "{to_path}"
!rsync --progress "{to_path}" "{REMOTE_URL}/"

# Zip the whole run folder (latest_subdir) and upload
run_name = os.path.basename(latest_subdir)  # e.g. 2026-01-31-11_panabat-1
zip_path = os.path.join("/content", f"{project}-{run_name}.zip")

print("zipping", latest_subdir, "->", zip_path)
!zip -r "{zip_path}" "{latest_subdir}"
!rsync --progress "{zip_path}" "{REMOTE_URL}/"

In [ ]:
!rsync --progress {to_path} {REMOTE_URL}/


### Training results plot

In [ ]:
# Change working directory to HOME
%cd {HOME}

# Display the training results plot (e.g. loss and metrics curves)
Image(filename=f'{latest_subdir}/results.png', width=1200)

### Sample batch of validation predictions

In [ ]:
# Change working directory to HOME
%cd {HOME}

# Display a sample batch of validation predictions (visual output of model)
Image(filename=f'{latest_subdir}/val_batch0_pred.jpg', width=600)

# 6) Validate Custom Model

This step runs **model validation** using the best trained checkpoint (`best.pt`) on the validation dataset defined in `data.yaml`. It evaluates the model's performance using standard YOLO metrics, such as:

- **mAP50**: mean Average Precision at IoU threshold 0.5
- **mAP50-95**: mean AP across IoU thresholds from 0.5 to 0.95
- **Precision & Recall** for each class

The validation results will be saved inside the specified project folder and include:

- A `results.png` file with training/validation curves
- A `confusion_matrix.png` for classification performance
- A `val_batch0_pred.jpg` showing predicted bounding boxes on a sample batch

You can use these visual and quantitative outputs to assess if the model generalizes well to unseen data. [link text](https://)

In [ ]:
# Change working directory to HOME
%cd {HOME}

# Display the training results plot (e.g. loss and metrics curves)
Image(filename=f'{latest_subdir}/results.png', width=1200)

# 7) Run Inference on Validation Images

This step performs **inference (prediction)** using the best trained YOLO model (`best.pt`) on the validation image set. It is useful to **visually inspect how the model performs** on real images after training.

What this does:

- Removes any existing `predict` folder to avoid clutter or overwriting previous predictions
- Runs YOLO in `predict` mode using:
  - The best model checkpoint
  - Images from the validation set
  - A low confidence threshold (`conf=0.1`) to allow more predictions for visual inspection
  - The specified image size (`imgsz`)
- Saves predicted images (with boxes, masks, or keypoints depending on the task) in a new folder under the project directory: `runs/predict`

This is especially helpful for qualitatively checking the model's detection performance, spotting failure cases, or selecting images for visualization or presentations.

In [ ]:
%cd {HOME}

best_ckpt = os.path.join(latest_subdir, "weights", "best.pt")
data_yaml = os.path.join(out_dir, "data.yaml")   # <-- fix: use out_dir

assert os.path.exists(best_ckpt), best_ckpt
assert os.path.exists(data_yaml), data_yaml

!yolo task={task} mode=val model="{best_ckpt}" data="{data_yaml}" project="{project}" imgsz={resolution} line_width=1

### Zip and Save Prediction Results

This step creates a ZIP archive of the prediction results generated in the previous step. The archive is saved in your home directory and named using the training subdirectory name (to make it easy to track which model it came from).

This makes it simple to download, share, or upload the predictions for external use (e.g., for presentations, manual inspection, or further analysis).

In [ ]:
# Change working directory to HOME
%cd {HOME}

best_ckpt   = os.path.join(latest_subdir, "weights", "best.pt")
valid_imgs  = os.path.join(out_dir, "valid", "images")   # <-- use out_dir (your dataset root)
pred_dir    = os.path.join(project, "predict")           # project/name

print("Model:", best_ckpt)
print("Source:", valid_imgs)
print("Will write to:", pred_dir)

# Remove any previous YOLO prediction results (safe delete)
if os.path.isdir(pred_dir):
    shutil.rmtree(pred_dir)

# Run YOLO prediction on validation images using the best model checkpoint
!yolo task={task} mode=predict model="{best_ckpt}" project="{project}" name="predict" conf=0.1 source="{valid_imgs}" save=True imgsz={resolution} line_width=1

# 10) Display Sample Predictions

This step randomly selects and displays 5 predicted images from the `predict` folder.

Each image includes the model's output (e.g., bounding boxes, masks, or keypoints) overlaid on the validation images.  
It provides a quick **visual inspection** of model performance across different examples.  

This qualitative check helps identify:
- How well the model localizes objects
- Possible false positives or negatives
- Class confusion or missed detections

In [ ]:
# Find newest predict* folder for this task/project
pred_root = f"/content/datasets/runs/{task}/{project}"
pred_candidates = [p for p in glob.glob(f"{pred_root}/predict*") if os.path.isdir(p)]
assert pred_candidates, f"No predict folder found in: {pred_root}"
pred_dir = max(pred_candidates, key=os.path.getmtime)

# Randomly select up to 5 predicted images
imgs = glob.glob(f"{pred_dir}/*.jpg")
assert imgs, f"No .jpg files found in: {pred_dir}"

k = min(5, len(imgs))
files = np.random.choice(imgs, size=k, replace=False)
print(files.shape)
print("Using:", pred_dir)

for image_path in files:
    display(Image(filename=image_path, height=600))
    print()

In [ ]:


# Wait for 30 seconds (e.g., to ensure all background tasks finish before disconnecting)
time.sleep(30)

# Gracefully disconnect the current Colab runtime session
runtime.unassign()


## 🏆 Congratulations

### Find more learning resources here

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.